<a href="https://colab.research.google.com/github/rskarbez/colab_notebooks/blob/main/CSE2DV_CSE5INV_Week_10_lab_worksheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE2DV/CSE5INV - Week 10 lab notebook

In this week's lab, we're going to learn to work with and to visualize network data. We're going to make use of the `pandas` package to load datasets, the `networkx` package to create and manipulate networks, and the `pyvis` package to visualize them.

From the `networkx` documentation:

> NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

From the `pyvis` documentation:

> The goal of this project is to build a python based approach to constructing and visualizing network graphs in the same space. A pyvis network can be customized on a per node or per edge basis. Nodes can be given colors, sizes, labels, and other metadata. Each graph can be interacted with, allowing the dragging, hovering, and selection of nodes and edges. Each graph’s layout algorithm can be tweaked as well to allow experimentation with rendering of larger graphs.



In [109]:
# pyvis is not available on Google Colab by default,
#   so we need to bring it in using conda
# NOTE: This should only take a minute, but please be patient!
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install pyvis

# Do the imports of pyvis, networkx, and pandas
from pyvis.network import Network
import networkx as nx
import pandas as pd

# Import IPython which is needed to display outputs in colab
import IPython

# Import shutil which is needed to unzip the pandas dataset
import shutil

✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.25.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['pyvis']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch 

# Manually editing Network() objects in pyvis.network

One way to bring in network data to visualize with `pyvis` is to manually edit a `pyvis.network.Network` object. We'll start off doing things that way. It's a bit tedious, but educational. First step is to instantiate a `Network` object:

`net = Network()`

Once your basic network has been created, you can add nodes to the network with the `Network.add_node()` function, either one at a time:

`net.add_node(1, label="Node 1") # node id = 1 and label = Node 1`

`net.add_node(2) # node id and label = 2`

or as a list:

`nodes = ["a", "b", "c", "d"]`

`net.add_nodes(nodes) # node ids and labels = ["a", "b", "c", "d"]`

`net.add_nodes("hello") # node ids and labels = ["h", "e", "l", "o"]`

With `add_node()` or `add_nodes()`, you can also add one or more keyword arguments associated with each node. The valid keywords (which are relatively few and hopefully self-explanatory) are 

`size, value, title, x, y, label, color`

This example employs all of them:

`g = Network()`

`g.add_nodes([1,2,3],`

`            value=[10, 100, 400],`

`            title=['I am node 1', 'node 2 here', 'and im node 3'],`

`            x=[21.4, 54.2, 11.2],`

`            y=[100.2, 23.54, 32.1],`

`            label=['NODE 1', 'NODE 2', 'NODE 3'],`

`            color=['#00ff1e', '#162347', '#dd4b39'])`


Once you have created nodes in your network, you can also add edges:

`net = Network()`

`net.add_node(0, label='a')`

`net.add_node(1, label='b')`

`net.add_edge(0, 1)`

Edges can have one keyword argument, `weight`, associated with them, as in:

`net.add_edge(0, 1, weight=.87)`

# Problem with interactive vis in Colab

So, now's the time where I tell you that this isn't as slick as it should be. If you were running this in a  Jupyter Notebook on your local computer, all you'd have to do is add `notebook=True` to the `Network()` call on the first line of these cells, and it would pop up an interactive visualization.

However, because of, like, "security" and "well-behaved webpages", that doesn't work in Colab. Here's what you do instead.


1.   Click the folder icon in the left-hand side of the window to bring up the file browser.
2.   Right-click on the html file that you want to run. The name of the file is the string argument that was passed to the `show()` function: For the first cell below, the corresponding HTML file is `3nodes.html`.
3.   Select `Download`.
4.   Once that file has downloaded, open the local copy in a web browser of your choice.

Voila! Interactive network visualization.

In [121]:
g = Network('500px', '500px')
g.add_nodes([1,2,3], value=[10, 100, 400],
                         title=['I am node 1', 'node 2 here', 'and im node 3'],
                         #x=[21.4, 54.2, 11.2],
                         #y=[100.2, 23.54, 32.1],
                         label=['NODE 1', 'NODE 2', 'NODE 3'],
                         color=['#00ff1e', '#162347', '#dd4b39'])
g.show('3nodes.html')

# Using the `networkx` package to create networks

For creating and editing networks, the `networkx` package is easier and more fully-featured than the `pyvis.Network` module. Luckily, it is possible to use the `networkx` package to create your network and use the handy pyvis helper function `.from_nx()` to convert the `networkx` graph to a `pyvis Network`. See the below example, which generates the output visualization in the `nx.html` file.

In [107]:
nx_graph = nx.cycle_graph(10)
nx_graph.nodes[1]['title'] = 'Number 1'
nx_graph.nodes[1]['group'] = 1
nx_graph.nodes[3]['title'] = 'I belong to a different group!'
nx_graph.nodes[3]['group'] = 10
nx_graph.add_node(20, size=20, title='couple', group=2)
nx_graph.add_node(21, size=15, title='couple', group=2)
nx_graph.add_edge(20, 21, weight=5)
nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
nt = Network('500px', '500px')
# populates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.show('nx.html')

# Importing network data using pandas

In [ ]:
# Download and extract the dataset
!wget https://melaniewalsh.github.io/Intro-Cultural-Analytics/_downloads/1d4d9192fd40b1204b4d790892ce8d5a/game-of-thrones-characters.zip
shutil.unpack_archive('game-of-thrones-characters.zip')

In [114]:
got_net = Network(height="750px", width="100%")

# set the physics layout of the network
got_net.barnes_hut()
got_data = pd.read_csv('/content/game-of-thrones/got-edges.csv')

got_nx = nx.from_pandas_edgelist(got_data, 'Source', 'Target', 'Weight')
got_net.from_nx(got_nx)

got_net.show('got-1.html')

# Alternative Network Visualizations with Bokeh

[Melanie Walsh](https://melaniewalsh.org/) has created an e-book on [Introduction to Cultural Analytics & Python](https://melaniewalsh.github.io/Intro-Cultural-Analytics/welcome.html). In it, she uses this same Game of Thrones dataset for an extensive discussion of [network visualization using Bokeh](https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html). I encourage you to walk through that notebook. Then, once you've taken a look at it, try to replicate some of the same visualization techniques in our `pyvis.Network` framework.